# **Complete END to END GitHub Bugs Prediction**

![Minion](https://www.aismartz.com/blog/wp-content/uploads/2019/11/Electronic-Design-Automation-data-science-model.jpg)

## **Overview Of The Problem**
In this problem, we are challenging the machine learning community to come up with an algorithm that can predict the bugs, features, and questions based on GitHub titles and the text body. With text data, there can be a lot of challenges especially when the dataset is big.  
With text data, there can be a lot of challenges especially when the dataset is big. Analyzing such a dataset requires a lot to be taken into account mainly due to the preprocessing involved to represent raw text and make them machine-understandable. Usually, we stem and lemmatize the raw information and then represent it using TF-IDF, Word Embeddings, etc.
However, provided the state-of-the-art NLP models such as Transformer based **BERT** models one can skip the manual feature engineering like **TF-IDF** and **Count Vectorizers**. In this short span of time, we would encourage you to leverage the ImageNet moment (Transfer Learning) in NLP using various pre-trained models.


# <center><span style="color:blue"> **Table of content** </span></center>

1. <span style="color:purple"> **Dataset Description**</span>.
2. <span style="color:purple"> **Attribute Description**</span>.
3. <span style="color:purple"> **Loading Dataset**</span>.
4. <span style="color:purple"> **Loading Libraries**</span>.
5. <span style="color:purple"> **Columns Exploration**</span>.
6. <span style="color:purple"> **Exploring Shape of Dataset**</span>.
7. <span style="color:purple"> **Statistical Analysis-I**</span>.

       


## **Dataset Description**
- Train.json - 150000 rows x 3 columns (Includes label Column as Target variable)
- Test.json - 30000 rows x 2 columns
- Train_extra.json - 300000 rows x 3 columns (Includes label Column as Target variable)
- Provided solely for training purposes, can be appended in the train.json for training the    model
- Sample Submission.csv - Please check the Evaluation section for more details on how to generate a valid submission

## **Attribute Description:** 
* Title - the title of the GitHub bug, feature, question
* Body - the body of the GitHub bug, feature, question
* Label - Represents various classes of Labels

    1. Bug - 0
    2. Feature - 1
    3. Question - 2



## **Loading Libraries**

In [ ]:
from bs4 import BeautifulSoup
from collections import Counter,defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import plotly.graph_objs as go
import plotly.offline as py
from plotly import tools
py.init_notebook_mode(connected=True)
import re
import seaborn as sns
import string
from unidecode import unidecode
from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

## **Loading Dataset**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df= pd.read_json("/kaggle/input/github-bugs-prediction/embold_train.json").reset_index(drop=True)
train_df.head()

- ## **Columns Exploration**

The "tiltle, and body" column contains the textual information(input features) and the "label" column contains the output labels. The task of any classifier is to correctly predict;
        -  Bug - 0
        -  Feature - 1
        -  Question - 2
That given any "label" or textual column. Hence we have to apply our data cleaning, transformation steps to the "title and body" column.

- ## **Exploring Shape of dataset**

In [ ]:
len(train_df),train_df.index.shape[-1]

In [ ]:
## Assess the shape of the data
print("The Shape of the Dataset".format(),train_df.shape)

In [ ]:
print('Number of data points : ', train_df.shape[0])
print('Number of features : ', train_df.shape[1])
print('Features : ', train_df.columns.values)

In [ ]:
train_df.info()

In [ ]:
#checking null values
train_df.isnull().sum()

In [ ]:
#checking values of label column
train_df['label'].value_counts()

![Analysis](https://learn.g2.com/hs-fs/hubfs/What%20is%20Statistical%20Analysis.png?width=900&name=What%20is%20Statistical%20Analysis.png)

- ### **Statistical Analysis-I**
This is the start of the analysis phase where we will first check the amount of data present in either of the labels. We will follow this up with some pictorial representations related to the words and frequency mappings.

In [ ]:
#The describe() function is used to generate descriptive statistics that summarize the central tendency, 
#dispersion and shape of a dataset’s distribution, excluding NaN values.
train_df.describe(include=['O'])  #include=['O'] for Object 

In [ ]:
#include='all' its include all details
train_df.describe(include='all')

In [ ]:
#checking percentage 
(train_df.label.value_counts(normalize=True).sort_index())*100

## **Hypothesize**
* Title columns have some duplicate. As 149677 is unique out of 150000
        "add unit tests" most repeative sentence in title
* Body have no duplicate.
* Label have three catergories
        0    44.551333 %
        1    46.070667 %
        2     9.378000 %
         

### **Pictorial representation of Label**

In [ ]:
#Count of all Labels

count=train_df['label'].value_counts()
print('Total Counts of all sets\n', count)

print("==============")
#Creating a function to plot the counts using matplotlib
def plot_counts(count_Bug,count_Feature,count_Question):
    plt.rcParams['figure.figsize']=(6,6)
    plt.bar(0,count_Bug,width=0.6,label='Bugs',color='Green')
    plt.legend()
    plt.bar(1,count_Feature,width=0.6,label='Features',color='Red')
    plt.legend()
    plt.bar(2,count_Question,width=0.6,label='Questions',color='Black')
    plt.legend()
    plt.ylabel('Count of Label')
    plt.xlabel('Types of Label')
    plt.show()
    
count_Bug=train_df[train_df['label']== 0]
count_Feature=train_df[train_df['label']==1]
count_Question=train_df[train_df['label']==2]
plot_counts(len(count_Bug),len(count_Feature),len(count_Question))

- ## **Hypothesize**
Here you can we balance between Bugs and Features class.

- ## **Filtering train_data based on each unique label category** 
        1. Bug - 0
        2. Feature - 1
        3. Question - 2


In [ ]:
count_Bug=train_df[train_df['label']== 0]
count_Feature=train_df[train_df['label']== 1]
count_Question=train_df[train_df['label']== 2]

In [ ]:
count_body_Bug = count_Bug.body.str.split().apply(lambda w : len(w)).sort_values(ascending=True)
count_body_Feature = count_Feature.body.str.split().apply(lambda w : len(w)).sort_values(ascending=True)
count_body_Question = count_Question.body.str.split().apply(lambda w : len(w)).sort_values(ascending=True)

### Pictorial representation of Word in each Label Catergory

In [ ]:
# Creating a Generic count_plot function with Seaborn

def plot_count_dist(count_Bug,count_Feature,count_Question,title_1,title_2,title_3,subtitle):
    fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(18,6))
    sns.distplot(count_Bug,ax=ax1,color='r')
    ax1.set_title(title_1)
    sns.distplot(count_Feature,ax=ax2,color='g')
    ax2.set_title(title_2)
    sns.distplot(count_Question,ax=ax3,color='b')
    ax3.set_title(title_3)
    fig.suptitle(subtitle)
    plt.show()
    
plot_count_dist(count_body_Bug,count_body_Feature,count_body_Question,'Bug','Feature','Question','Body Data Word Count Aanalysis')


- ### **Generating WordCloud across different label categories.**

In [ ]:
#Creating a generic function for generating WordCloud across different label categories.

def generate_wordcloud(df,col,i,label):
    
    data = df[df.label == i][col].values
    
    wc = WordCloud(stopwords=STOPWORDS, background_color='black',
                   max_words=10000, min_font_size=6, min_word_length=1)
    wc.generate(' '.join(data))
    
    plt.figure(figsize=(15,15))
    plt.title('WordCloud for {}'.format(label), fontsize = 24)
    plt.imshow(wc)
    plt.axis("off")
    plt.show()
    


labels = np.unique(train_df.label.values)
label_names = ['Bug','Feature','Question']
for i in labels:
    generate_wordcloud(train_df,'body',i,label_names[i])